# 추천 시스템 (Recommender System)

컨텐츠 기반 필터링 (content-based filtering) - 사용자의 이전 행동 명시적인 피드백을 통해서 사용자가 좋아하는 것인가 유사한가를 고려해서 추천

협업 필터링 (collarborativefiltering) - 사용자와 항목 간의 유사성을 동시에 사용해 추천

### Surprise
- 추천 시스템 개발을 위한 라이브러리
- 다양한 모델과 데이터 제공
- scikit-learn과 유사한 사용 방법

간단한 surprise 실습

In [1]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [2]:
data = Dataset.load_builtin("ml-100k", prompt = False)
data.raw_ratings[:10] 

# 사용자가 어떤 영화에 대해서 rating을 몇 점을 줬는지가 중요한 데이터
# 유저 / 아이템(영화) / rating 점수 / 아이디

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\ASIA-08/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [3]:
model = SVD()

In [4]:
cross_validate(model, data, measures = ['rmse', 'mae'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9319  0.9341  0.9436  0.9320  0.9369  0.9357  0.0044  
MAE (testset)     0.7337  0.7362  0.7430  0.7365  0.7400  0.7379  0.0033  
Fit time          3.25    3.40    3.26    3.27    3.35    3.31    0.06    
Test time         0.10    0.09    0.12    0.12    0.12    0.11    0.01    


{'test_rmse': array([0.9319201 , 0.93411902, 0.94364251, 0.93202831, 0.93689003]),
 'test_mae': array([0.73370031, 0.73621104, 0.74300663, 0.73651451, 0.73999692]),
 'fit_time': (3.251962423324585,
  3.404677391052246,
  3.2594852447509766,
  3.2672054767608643,
  3.345810651779175),
 'test_time': (0.0960545539855957,
  0.09477353096008301,
  0.11668705940246582,
  0.11828160285949707,
  0.12102150917053223)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

- 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
    - ex) 내가 지금까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천

- 유사도를 기반으로 추천

- 장단점
    - 장점
        - 많은 수의 사용자를 대상으로 쉽게 확장 가능
        - 사용자가 관심을 갖지 않던 상품 추천 가능
    - 단점
        - 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
        - 사용자의 기존 관심사항을 기반으로만 추천 가능

In [5]:
import numpy as np
from surprise import Dataset

- 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
- 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [6]:
data = Dataset.load_builtin('ml-100k', prompt = False) # 영화
raw_data = np.array(data.raw_ratings, dtype = int)

In [7]:
# 0부터 시작할 수 있도록
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [8]:
# 인접 행렬 크기 구하기
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])

# 왜 + 1 하는 거지
shape = (n_users + 1, n_movies + 1)
shape

(943, 1682)

In [9]:
# 인접 행렬 만들기 - 특정 값에 1을 넣어줌(1이 있는 곳이 데이터가 있는 것)
# 관람의 유무
adj_matrix = np.ndarray(shape, dtype = int)

for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = 1.

adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [11]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    # 나 자신이 아닌 아이디와 similarity 비교
    if my_id != user_id:
        similarity = np.dot(my_vector, user_vector)
        if similarity > best_match:
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 183, Best Match ID: 275


In [12]:
recommend_list = []

for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0:
        recommend_list.append(i)

print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

- 유클리드 거리를 사용해 추천

- 거리가 가까울수록(값이 작을수록) 나와 유사한 사용자

In [14]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    # 나 자신이 아닌 아이디와 similarity 비교
    if my_id != user_id:
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 14.832396974191326, Best Match ID: 737


In [15]:
recommend_list = []

for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0:
        recommend_list.append(i)

print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


- 코사인 유사도를 사용해 추천

- 두 벡터가 이루고 있는 각을 계산

In [16]:
def compute_cos_similarity(v1, v2):
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1, v2)
    return dot / (norm1 * norm2)

In [17]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    # 나 자신이 아닌 아이디와 similarity 비교
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.5278586163659506, Best Match ID: 915


In [18]:
recommend_list = []

for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0:
        recommend_list.append(i)

print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [19]:
# 평가 점수
adj_matrix = np.ndarray(shape, dtype=int)

for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating

adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [22]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    # 나 자신이 아닌 아이디와 similarity 비교
    if my_id != user_id:
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 55.06359959174482, Best Match ID: 737


In [23]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    # 나 자신이 아닌 아이디와 similarity 비교
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.569065731527988, Best Match ID: 915


# 협업 필터링 (Collaborative Filtering)
    - 사용자와 항목의 유사성을 동시에 고려해 추천
    - 기존에 내 관심사가 아닌 항목이라도 추천 가능
    - 자동으로 임베딩 학습 가능
    
    - 장단점
        - 장점
            - 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요없다
            - 기존의 관심사가 아니더라도 추천 가능

        - 단점
            - 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
            - 추가 특성을 사용하기 어려움